In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sklearn.linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix,r2_score,mean_squared_error,mean_absolute_error
from sklearn import neural_network
from sklearn.model_selection import GridSearchCV

import sys
!{sys.executable} -m pip install lightgbm
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

In [2]:
df = pd.read_csv("only_genres_keywords.csv")

In [3]:
df.shape

(3166, 55)

In [4]:
columns_to_normalize = ['vote_count','budget', 'revenue']
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

In [5]:
df_corr = df.drop(columns = ['title'])
cor_matrix = df_corr.corr(method="spearman").round(2)
cor_matrix.style.background_gradient(cmap="jet")

,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,genre_Crime,genre_Drama,genre_Family,genre_Fantasy,genre_History,genre_Horror,genre_Music,genre_Mystery,genre_Romance,genre_Science_Fiction,genre_Thriller,genre_War,keyword_3d,keyword_aftercreditsstinger,keyword_alien,keyword_based_on_comic_book,keyword_based_on_novel,keyword_biography,keyword_daughter,keyword_duringcreditsstinger,keyword_dying_and_death,keyword_dystopia,keyword_family,keyword_father_son_relationship,keyword_friendship,keyword_high_school,keyword_independent_film,keyword_london_england,keyword_los_angeles,keyword_love,keyword_murder,keyword_musical,keyword_new_york,keyword_nudity,keyword_police,keyword_prison,keyword_remake,keyword_revenge,keyword_sequel,keyword_sex,keyword_sport,keyword_superhero,keyword_suspense,keyword_teenager,keyword_violence,keyword_woman_director,budget,revenue,vote_average,vote_count
genre_Action,1.000000,0.360000,-0.100000,-0.190000,0.140000,-0.230000,-0.130000,0.060000,0.000000,-0.100000,-0.090000,-0.080000,-0.220000,0.250000,0.270000,0.060000,0.080000,0.030000,0.060000,0.180000,-0.050000,-0.060000,-0.010000,0.000000,-0.030000,0.190000,-0.050000,-0.040000,-0.060000,-0.060000,-0.080000,0.040000,0.040000,-0.070000,-0.010000,-0.080000,-0.000000,-0.030000,0.020000,0.010000,-0.030000,0.110000,0.090000,-0.070000,-0.040000,0.200000,0.010000,-0.070000,0.100000,-0.090000,0.310000,0.200000,-0.140000,0.170000
genre_Adventure,0.360000,1.000000,0.180000,-0.070000,-0.130000,-0.240000,0.240000,0.230000,-0.020000,-0.150000,-0.070000,-0.070000,-0.130000,0.220000,-0.030000,0.020000,0.160000,0.100000,0.090000,0.170000,0.010000,-0.060000,-0.000000,0.050000,0.010000,0.070000,-0.020000,-0.010000,-0.000000,-0.050000,-0.080000,0.050000,-0.040000,-0.040000,-0.050000,-0.010000,0.000000,-0.040000,-0.050000,-0.040000,-0.040000,0.000000,0.090000,-0.050000,-0.050000,0.140000,-0.030000,-0.040000,-0.030000,-0.060000,0.340000,0.280000,-0.060000,0.200000
genre_Animation,-0.100000,0.180000,1.000000,0.100000,-0.110000,-0.180000,0.580000,0.120000,-0.050000,-0.090000,0.030000,-0.070000,-0.090000,-0.030000,-0.160000,-0.040000,0.160000,0.080000,0.050000,-0.010000,-0.020000,-0.040000,-0.010000,0.120000,-0.020000,-0.010000,0.050000,0.020000,0.080000,-0.040000,-0.050000,-0.020000,-0.030000,0.010000,-0.050000,0.070000,-0.030000,-0.030000,-0.040000,-0.020000,-0.030000,-0.030000,0.040000,-0.030000,-0.020000,-0.020000,-0.040000,-0.040000,-0.050000,0.020000,0.190000,0.180000,0.050000,0.110000
genre_Comedy,-0.190000,-0.070000,0.100000,1.000000,-0.100000,-0.230000,0.210000,0.030000,-0.130000,-0.170000,0.050000,-0.180000,0.190000,-0.140000,-0.390000,-0.110000,-0.030000,0.070000,-0.040000,-0.040000,-0.070000,-0.070000,-0.010000,0.170000,-0.060000,-0.120000,0.040000,-0.010000,0.060000,0.090000,0.060000,0.000000,0.010000,0.010000,-0.110000,0.030000,0.030000,-0.030000,-0.030000,-0.010000,-0.060000,-0.080000,-0.010000,0.030000,0.020000,-0.060000,-0.100000,0.030000,-0.100000,0.070000,-0.050000,-0.040000,-0.200000,-0.110000
genre_Crime,0.140000,-0.130000,-0.110000,-0.100000,1.000000,0.080000,-0.140000,-0.140000,-0.050000,-0.110000,-0.050000,0.110000,-0.130000,-0.140000,0.300000,-0.070000,-0.060000,-0.040000,-0.060000,-0.020000,-0.000000,-0.010000,0.010000,-0.040000,-0.040000,-0.080000,-0.020000,-0.010000,-0.020000,-0.030000,0.010000,0.020000,0.080000,-0.020000,0.160000,-0.050000,0.050000,0.060000,0.160000,0.060000,0.000000,0.110000,-0.020000,0.020000,-0.060000,-0.010000,0.110000,-0.040000,0.090000,-0.060000,-0.020000,-0.070000,0.050000,-0.010000
genre_Drama,-0.230000,-0.240000,-0.180000,-0.230000,0.080000,1.000000,-0.180000,-0.150000,0.200000,-0.210000,0.070000,0.030000,0.200000,-0.200000,-0.040000,0.130000,-0.090000,-0.140000,-0.100000,-0.100000,0.070000,0.150000,0.020000,-0.150000,0.020000,-0.120000,0.010000,0.050000,0.030000,-0.040000,0.090000,-0.020000,-0.000000,0.080000,0.000000,0.040000,0.010000,0.020000,-0.000000,0.020000,-0.030000,-0.040000,-0.130000,0.040000,0.080000,-0.090

In [6]:
X=df.values.tolist();
y=[];
for row in X:
    y.append(float(row[53]));
    del row[53]
    del row[52]
X=pd.Series(X);
y=pd.Series(y);

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True);

X_train_list=X_train.values.tolist();
y_train_list=y_train.values.tolist();
X_test_list=X_test.values.tolist();
y_test_list=y_test.values.tolist();

# Modellerin Eğitim kısmı

## Accuracy Within Threshold

In [7]:
def AWT(y_pred,y_true,threshold):
    dummy = list()
    for i in range(len(y_pred)):
        dif = abs(y_pred[i] - y_true[i])
        if dif <= threshold:
            dummy.append(1)
        else: dummy.append(0)
    return dummy.count(1) / len(dummy)

## Lineer Regresyon

In [8]:
deneme = sklearn.linear_model.LinearRegression()

In [9]:
deneme.fit(X_train_list, y_train_list) 

LinearRegression()

In [10]:
y_pred_lr = deneme.predict(X_test_list)

In [11]:
y_pred_lr_t = deneme.predict(X_train_list)

In [12]:
mse_lr = mean_squared_error(y_test_list, y_pred_lr)
r2_lr = r2_score(y_test_list, y_pred_lr)
mae_lr = mean_absolute_error(y_test_list, y_pred_lr)
AWT_1_lr = AWT(y_test_list, y_pred_lr,0.1)
AWT_3_lr = AWT(y_test_list, y_pred_lr,0.3)
AWT_5_lr = AWT(y_test_list, y_pred_lr,0.5)
AWT_7_lr = AWT(y_test_list, y_pred_lr,0.7)
AWT_9_lr = AWT(y_test_list, y_pred_lr,0.9)
lr_data = [mse_lr,r2_lr,mae_lr]
lr_data2 = [AWT_1_lr,AWT_3_lr,AWT_5_lr,AWT_7_lr,AWT_9_lr]

In [13]:
mse_lr_t = mean_squared_error(y_train_list, y_pred_lr_t)
r2_lr_t = r2_score(y_train_list, y_pred_lr_t)
mae_lr_t = mean_absolute_error(y_train_list, y_pred_lr_t)
AWT_1_lr_t = AWT(y_train_list, y_pred_lr_t,0.1)
AWT_3_lr_t = AWT(y_train_list, y_pred_lr_t,0.3)
AWT_5_lr_t = AWT(y_train_list, y_pred_lr_t,0.5)
AWT_7_lr_t = AWT(y_train_list, y_pred_lr_t,0.7)
AWT_9_lr_t = AWT(y_train_list, y_pred_lr_t,0.9)
lr_data_t = [mse_lr_t,r2_lr_t,mae_lr_t]
lr_data2_t = [AWT_1_lr_t,AWT_3_lr_t,AWT_5_lr_t,AWT_7_lr_t,AWT_9_lr_t]

In [14]:
print("MSE value:",mse_lr)
print("R^2 score:",r2_lr)
print("MAE value:", mae_lr)
print("AWT value for 0.1 threshold:",AWT_1_lr)
print("AWT value for 0.3 threshold:",AWT_3_lr)
print("AWT value for 0.5 threshold:",AWT_5_lr)
print("AWT value for 0.7 threshold:",AWT_7_lr)
print("AWT value for 0.9 threshold:",AWT_9_lr)

MSE value: 0.398686452816672
R^2 score: 0.4254475046343752
MAE value: 0.4827557587933083
AWT value for 0.1 threshold: 0.14353312302839116
AWT value for 0.3 threshold: 0.4148264984227129
AWT value for 0.5 threshold: 0.613564668769716
AWT value for 0.7 threshold: 0.7570977917981072
AWT value for 0.9 threshold: 0.8659305993690851


In [15]:
print("MSE value:",mse_lr_t)
print("R^2 score:",r2_lr_t)
print("MAE value:", mae_lr_t)
print("AWT value for 0.1 threshold:",AWT_1_lr_t)
print("AWT value for 0.3 threshold:",AWT_3_lr_t)
print("AWT value for 0.5 threshold:",AWT_5_lr_t)
print("AWT value for 0.7 threshold:",AWT_7_lr_t)
print("AWT value for 0.9 threshold:",AWT_9_lr_t)

MSE value: 0.38018305506657973
R^2 score: 0.4539445690039613
MAE value: 0.47732319118623284
AWT value for 0.1 threshold: 0.13507109004739337
AWT value for 0.3 threshold: 0.396129541864139
AWT value for 0.5 threshold: 0.6105845181674565
AWT value for 0.7 threshold: 0.773696682464455
AWT value for 0.9 threshold: 0.8692733017377567


## MLP Regressor

In [16]:
deneme2 = neural_network.MLPRegressor(hidden_layer_sizes=(100,100),activation = 'tanh',solver = 'sgd',learning_rate = 'adaptive' ,max_iter = 2000,alpha = 0.0011549425983027062)

In [17]:
deneme2.fit(X_train_list, y_train_list) 

MLPRegressor(activation='tanh', alpha=0.0011549425983027062,
             hidden_layer_sizes=(100, 100), learning_rate='adaptive',
             max_iter=2000, solver='sgd')

In [18]:
y_pred_mlp = deneme2.predict(X_test_list)

In [19]:
y_pred_mlp_t = deneme2.predict(X_train_list)

In [20]:
mse_mlp = mean_squared_error(y_test_list, y_pred_mlp)
r2_mlp = r2_score(y_test_list, y_pred_mlp)
mae_mlp = mean_absolute_error(y_test_list, y_pred_mlp)
AWT_1_mlp = AWT(y_test_list, y_pred_mlp,0.1)
AWT_3_mlp = AWT(y_test_list, y_pred_mlp,0.3)
AWT_5_mlp = AWT(y_test_list, y_pred_mlp,0.5)
AWT_7_mlp = AWT(y_test_list, y_pred_mlp,0.7)
AWT_9_mlp = AWT(y_test_list, y_pred_mlp,0.9)
mlp_data = [mse_mlp,r2_mlp,mae_mlp]
mlp_data2 = [AWT_1_mlp,AWT_3_mlp,AWT_5_mlp,AWT_7_mlp,AWT_9_mlp]

In [21]:
mse_mlp_t = mean_squared_error(y_train_list, y_pred_mlp_t)
r2_mlp_t = r2_score(y_train_list, y_pred_mlp_t)
mae_mlp_t = mean_absolute_error(y_train_list, y_pred_mlp_t)
AWT_1_mlp_t = AWT(y_train_list, y_pred_mlp_t,0.1)
AWT_3_mlp_t = AWT(y_train_list, y_pred_mlp_t,0.3)
AWT_5_mlp_t = AWT(y_train_list, y_pred_mlp_t,0.5)
AWT_7_mlp_t = AWT(y_train_list, y_pred_mlp_t,0.7)
AWT_9_mlp_t = AWT(y_train_list, y_pred_mlp_t,0.9)
mlp_data_t = [mse_mlp_t,r2_mlp_t,mae_mlp_t]
mlp_data2_t = [AWT_1_mlp_t,AWT_3_mlp_t,AWT_5_mlp_t,AWT_7_mlp_t,AWT_9_mlp_t]

In [22]:
print("MSE value:",mse_mlp)
print("R^2 score:",r2_mlp)
print("MAE value:", mae_mlp)
print("AWT value for 0.1 threshold:",AWT_1_mlp)
print("AWT value for 0.3 threshold:",AWT_3_mlp)
print("AWT value for 0.5 threshold:",AWT_5_mlp)
print("AWT value for 0.7 threshold:",AWT_7_mlp)
print("AWT value for 0.9 threshold:",AWT_9_mlp)

MSE value: 0.3914818399309437
R^2 score: 0.4358301707179453
MAE value: 0.47930782043609504
AWT value for 0.1 threshold: 0.14037854889589904
AWT value for 0.3 threshold: 0.42586750788643535
AWT value for 0.5 threshold: 0.613564668769716
AWT value for 0.7 threshold: 0.7539432176656151
AWT value for 0.9 threshold: 0.8659305993690851


In [23]:
print("MSE value:",mse_mlp_t)
print("R^2 score:",r2_mlp_t)
print("MAE value:", mae_mlp_t)
print("AWT value for 0.1 threshold:",AWT_1_mlp_t)
print("AWT value for 0.3 threshold:",AWT_3_mlp_t)
print("AWT value for 0.5 threshold:",AWT_5_mlp_t)
print("AWT value for 0.7 threshold:",AWT_7_mlp_t)
print("AWT value for 0.9 threshold:",AWT_9_mlp_t)

MSE value: 0.3698188792313976
R^2 score: 0.4688305941099671
MAE value: 0.46954148821697045
AWT value for 0.1 threshold: 0.13902053712480253
AWT value for 0.3 threshold: 0.40639810426540285
AWT value for 0.5 threshold: 0.6149289099526066
AWT value for 0.7 threshold: 0.7788309636650869
AWT value for 0.9 threshold: 0.877172195892575


## LightGBM

In [24]:
params = {
    'objective': 'regression',
    'metric': 'l2',  # Mean Squared Error
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}
X_train_np = np.array(X_train_list)
X_test_np = np.array(X_test_list)
y_train_np = np.array(y_train_list)
y_test_np = np.array(y_test_list)

train_data = lgb.Dataset(X_train_np, label=y_train_np)
test_data = lgb.Dataset(X_test_np, label=y_test_np, reference=train_data)

num_round = 100
bst = lgb.train(params,train_data, num_round, valid_sets=test_data)

y_pred_gbm = bst.predict(X_test_np, num_iteration=bst.best_iteration)
y_pred_gbm_t = bst.predict(X_train_np, num_iteration=bst.best_iteration)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004400 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 811
[LightGBM] [Info] Number of data points in the train set: 2532, number of used features: 53
[LightGBM] [Info] Start training from score 6.350948


In [25]:
mse_gbm = mean_squared_error(y_test_list, y_pred_gbm)
r2_gbm = r2_score(y_test_list, y_pred_gbm)
mae_gbm = mean_absolute_error(y_test_list, y_pred_gbm)
AWT_1_gbm = AWT(y_test_list, y_pred_gbm,0.1)
AWT_3_gbm = AWT(y_test_list, y_pred_gbm,0.3)
AWT_5_gbm = AWT(y_test_list, y_pred_gbm,0.5)
AWT_7_gbm = AWT(y_test_list, y_pred_gbm,0.7)
AWT_9_gbm = AWT(y_test_list, y_pred_gbm,0.9)
gbm_data = [mse_gbm,r2_gbm,mae_gbm]
gbm_data2 = [AWT_1_gbm,AWT_3_gbm,AWT_5_gbm,AWT_7_gbm,AWT_9_gbm]

In [26]:
mse_gbm_t = mean_squared_error(y_train_list, y_pred_gbm_t)
r2_gbm_t = r2_score(y_train_list, y_pred_gbm_t)
mae_gbm_t = mean_absolute_error(y_train_list, y_pred_gbm_t)
AWT_1_gbm_t = AWT(y_train_list, y_pred_gbm_t,0.1)
AWT_3_gbm_t = AWT(y_train_list, y_pred_gbm_t,0.3)
AWT_5_gbm_t = AWT(y_train_list, y_pred_gbm_t,0.5)
AWT_7_gbm_t = AWT(y_train_list, y_pred_gbm_t,0.7)
AWT_9_gbm_t = AWT(y_train_list, y_pred_gbm_t,0.9)
gbm_data_t = [mse_gbm_t,r2_gbm_t,mae_gbm_t]
gbm_data2_t = [AWT_1_gbm_t,AWT_3_gbm_t,AWT_5_gbm_t,AWT_7_gbm_t,AWT_9_gbm_t]

In [27]:
print("MSE value:",mse_gbm)
print("R^2 score:",r2_gbm)
print("MAE value:", mae_gbm)
print("HRA value for 0.1 threshold:",AWT_1_gbm)
print("HRA value for 0.3 threshold:",AWT_3_gbm)
print("HRA value for 0.5 threshold:",AWT_5_gbm)
print("HRA value for 0.7 threshold:",AWT_7_gbm)
print("HRA value for 0.9 threshold:",AWT_9_gbm)

MSE value: 0.32909914579031274
R^2 score: 0.5257307237287379
MAE value: 0.4396644410361559
HRA value for 0.1 threshold: 0.1498422712933754
HRA value for 0.3 threshold: 0.4242902208201893
HRA value for 0.5 threshold: 0.6750788643533123
HRA value for 0.7 threshold: 0.7949526813880127
HRA value for 0.9 threshold: 0.8927444794952681


In [28]:
print("MSE value:",mse_gbm_t)
print("R^2 score:",r2_gbm_t)
print("MAE value:", mae_gbm_t)
print("AWT value for 0.1 threshold:",AWT_1_gbm_t)
print("AWT value for 0.3 threshold:",AWT_3_gbm_t)
print("AWT value for 0.5 threshold:",AWT_5_gbm_t)
print("AWT value for 0.7 threshold:",AWT_7_gbm_t)
print("AWT value for 0.9 threshold:",AWT_9_gbm_t)

MSE value: 0.21201280844434384
R^2 score: 0.695486834699978
MAE value: 0.35589611667979926
AWT value for 0.1 threshold: 0.19233807266982622
AWT value for 0.3 threshold: 0.5138230647709321
AWT value for 0.5 threshold: 0.7503949447077409
AWT value for 0.7 threshold: 0.8846761453396524
AWT value for 0.9 threshold: 0.9498420221169036


## SVR

In [29]:
svr1 = SVR(kernel = 'linear',C = 10,epsilon = 0.2)

In [30]:
svr1.fit(X_train_list, y_train_list)

SVR(C=10, epsilon=0.2, kernel='linear')

In [31]:
y_pred_svr = svr1.predict(X_test_list)

In [32]:
y_pred_svr_t = svr1.predict(X_train_list)

In [33]:
mse_svr = mean_squared_error(y_test_list, y_pred_svr)
r2_svr = r2_score(y_test_list, y_pred_svr)
mae_svr = mean_absolute_error(y_test_list, y_pred_svr)
AWT_1_svr = AWT(y_test_list, y_pred_svr,0.1)
AWT_3_svr = AWT(y_test_list, y_pred_svr,0.3)
AWT_5_svr = AWT(y_test_list, y_pred_svr,0.5)
AWT_7_svr = AWT(y_test_list, y_pred_svr,0.7)
AWT_9_svr = AWT(y_test_list, y_pred_svr,0.9)
svr_data = [mse_svr,r2_svr,mae_svr]
svr_data2 = [AWT_1_svr,AWT_3_svr,AWT_5_svr,AWT_7_svr,AWT_9_svr]

In [34]:
mse_svr_t = mean_squared_error(y_train_list, y_pred_svr_t)
r2_svr_t = r2_score(y_train_list, y_pred_svr_t)
mae_svr_t = mean_absolute_error(y_train_list, y_pred_svr_t)
AWT_1_svr_t = AWT(y_train_list, y_pred_svr_t,0.1)
AWT_3_svr_t = AWT(y_train_list, y_pred_svr_t,0.3)
AWT_5_svr_t = AWT(y_train_list, y_pred_svr_t,0.5)
AWT_7_svr_t = AWT(y_train_list, y_pred_svr_t,0.7)
AWT_9_svr_t = AWT(y_train_list, y_pred_svr_t,0.9)
svr_data_t = [mse_svr_t,r2_svr_t,mae_svr_t]
svr_data2_t = [AWT_1_svr_t,AWT_3_svr_t,AWT_5_svr_t,AWT_7_svr_t,AWT_9_svr_t]

In [35]:
print("MSE value:",mse_svr)
print("R^2 score:",r2_svr)
print("MAE value:", mae_svr)
print("AWT value for 0.1 threshold:",AWT_1_svr)
print("AWT value for 0.3 threshold:",AWT_3_svr)
print("AWT value for 0.5 threshold:",AWT_5_svr)
print("AWT value for 0.7 threshold:",AWT_7_svr)
print("AWT value for 0.9 threshold:",AWT_9_svr)

MSE value: 0.40138283722431434
R^2 score: 0.42156170821733907
MAE value: 0.4839693780692358
AWT value for 0.1 threshold: 0.1498422712933754
AWT value for 0.3 threshold: 0.40536277602523657
AWT value for 0.5 threshold: 0.613564668769716
AWT value for 0.7 threshold: 0.750788643533123
AWT value for 0.9 threshold: 0.8596214511041009


In [36]:
print("MSE value:",mse_svr_t)
print("R^2 score:",r2_svr_t)
print("MAE value:", mae_svr_t)
print("AWT value for 0.1 threshold:",AWT_1_svr_t)
print("AWT value for 0.3 threshold:",AWT_3_svr_t)
print("AWT value for 0.5 threshold:",AWT_5_svr_t)
print("AWT value for 0.7 threshold:",AWT_7_svr_t)
print("AWT value for 0.9 threshold:",AWT_9_svr_t)

MSE value: 0.38391762118977074
R^2 score: 0.44858062632738627
MAE value: 0.474586947002872
AWT value for 0.1 threshold: 0.13704581358609794
AWT value for 0.3 threshold: 0.41113744075829384
AWT value for 0.5 threshold: 0.6204581358609794
AWT value for 0.7 threshold: 0.7701421800947867
AWT value for 0.9 threshold: 0.8696682464454977


## Random Forest 

In [37]:
rf1 = RandomForestRegressor(random_state=42,bootstrap = True,max_depth = None,min_samples_leaf = 2,min_samples_split = 10,n_estimators = 100)

In [38]:
rf1.fit(X_train_list, y_train_list)

RandomForestRegressor(min_samples_leaf=2, min_samples_split=10, random_state=42)

In [39]:
y_pred_rf = rf1.predict(X_test_list)
y_pred_rf_t = rf1.predict(X_train_list)

In [40]:
mse_rf = mean_squared_error(y_test_list, y_pred_rf)
r2_rf = r2_score(y_test_list, y_pred_rf)
mae_rf = mean_absolute_error(y_test_list, y_pred_rf)
AWT_1_rf = AWT(y_test_list, y_pred_rf,0.1)
AWT_3_rf = AWT(y_test_list, y_pred_rf,0.3)
AWT_5_rf = AWT(y_test_list, y_pred_rf,0.5)
AWT_7_rf = AWT(y_test_list, y_pred_rf,0.7)
AWT_9_rf = AWT(y_test_list, y_pred_rf,0.9)
rf_data = [mse_rf,r2_rf,mae_rf]
rf_data2 = [AWT_1_rf,AWT_3_rf,AWT_5_rf,AWT_7_rf,AWT_9_rf]

In [41]:
mse_rf_t = mean_squared_error(y_train_list, y_pred_rf_t)
r2_rf_t = r2_score(y_train_list, y_pred_rf_t)
mae_rf_t = mean_absolute_error(y_train_list, y_pred_rf_t)
AWT_1_rf_t = AWT(y_train_list, y_pred_rf_t,0.1)
AWT_3_rf_t = AWT(y_train_list, y_pred_rf_t,0.3)
AWT_5_rf_t = AWT(y_train_list, y_pred_rf_t,0.5)
AWT_7_rf_t = AWT(y_train_list, y_pred_rf_t,0.7)
AWT_9_rf_t = AWT(y_train_list, y_pred_rf_t,0.9)
rf_data_t = [mse_rf_t,r2_rf_t,mae_rf_t]
rf_data2_t = [AWT_1_rf_t,AWT_3_rf_t,AWT_5_rf_t,AWT_7_rf_t,AWT_9_rf_t]

In [42]:
print("MSE value:",mse_rf)
print("R^2 score:",r2_rf)
print("MAE value:", mae_rf)
print("AWT value for 0.1 threshold:",AWT_1_rf)
print("AWT value for 0.3 threshold:",AWT_3_rf)
print("AWT value for 0.5 threshold:",AWT_5_rf)
print("AWT value for 0.7 threshold:",AWT_7_rf)
print("AWT value for 0.9 threshold:",AWT_9_rf)

MSE value: 0.3562155854420835
R^2 score: 0.4866528519894633
MAE value: 0.45408216044022126
AWT value for 0.1 threshold: 0.1719242902208202
AWT value for 0.3 threshold: 0.41324921135646686
AWT value for 0.5 threshold: 0.6451104100946372
AWT value for 0.7 threshold: 0.7823343848580442
AWT value for 0.9 threshold: 0.8753943217665615


In [43]:
print("MSE value:",mse_rf_t)
print("R^2 score:",r2_rf_t)
print("MAE value:", mae_rf_t)
print("AWT value for 0.1 threshold:",AWT_1_rf_t)
print("AWT value for 0.3 threshold:",AWT_3_rf_t)
print("AWT value for 0.5 threshold:",AWT_5_rf_t)
print("AWT value for 0.7 threshold:",AWT_7_rf_t)
print("AWT value for 0.9 threshold:",AWT_9_rf_t)

MSE value: 0.1464031204060038
R^2 score: 0.7897217723223748
MAE value: 0.29015881133689025
AWT value for 0.1 threshold: 0.2278830963665087
AWT value for 0.3 threshold: 0.6153238546603476
AWT value for 0.5 threshold: 0.8388625592417062
AWT value for 0.7 threshold: 0.933649289099526
AWT value for 0.9 threshold: 0.9711690363349131


### Test Results

In [44]:
mse_data = [mse_lr,mse_mlp,mse_gbm,mse_svr,mse_rf]
r2_data = [r2_lr,r2_mlp,r2_gbm,r2_svr,r2_rf]
mae_data = [mae_lr,mae_mlp,mae_gbm,mae_svr,mae_rf]
AWT_1_data = [AWT_1_lr,AWT_1_mlp,AWT_1_gbm,AWT_1_svr,AWT_1_rf]
AWT_3_data = [AWT_3_lr,AWT_3_mlp,AWT_3_gbm,AWT_3_svr,AWT_3_rf]
AWT_5_data = [AWT_5_lr,AWT_5_mlp,AWT_5_gbm,AWT_5_svr,AWT_5_rf]
AWT_7_data = [AWT_7_lr,AWT_7_mlp,AWT_7_gbm,AWT_7_svr,AWT_7_rf]
AWT_9_data = [AWT_9_lr,AWT_9_mlp,AWT_9_gbm,AWT_9_svr,AWT_9_rf]

In [45]:
data = [
    mse_data,
    r2_data,
    mae_data,
    AWT_1_data,
    AWT_3_data,
    AWT_5_data,
    AWT_7_data,
    AWT_9_data
]

table = pd.DataFrame(data, columns=['Lineer reg', 'MLP', 'LightGBM','SVR','RF'])

table.index = ['MSE', 'R^2', 'MAE','AWT 0.1','AWT 0.3','AWT 0.5','AWT 0.7','AWT 0.9']
table.head(10)

,Lineer reg,MLP,LightGBM,SVR,RF
MSE,0.398686,0.391482,0.329099,0.401383,0.356216
R^2,0.425448,0.435830,0.525731,0.421562,0.486653
MAE,0.482756,0.479308,0.439664,0.483969,0.454082
AWT 0.1,0.143533,0.140379,0.149842,0.149842,0.171924
AWT 0.3,0.414826,0.425868,0.424290,0.405363,0.413249
AWT 0.5,0.613565,0.613565,0.675079,0.613565,0.645110
AWT 0.7,0.757098,0.753943,0.794953,0.750789,0.782334
AWT 0.9,0.865931,0.865931,0.892744,0.859621,0.875394


### Training Results

In [46]:
mse_data_t = [mse_lr_t,mse_mlp_t,mse_gbm_t,mse_svr_t,mse_rf_t]
r2_data_t = [r2_lr_t,r2_mlp_t,r2_gbm_t,r2_svr_t,r2_rf_t]
mae_data_t = [mae_lr_t,mae_mlp_t,mae_gbm_t,mae_svr_t,mae_rf_t]
AWT_1_data_t = [AWT_1_lr_t,AWT_1_mlp_t,AWT_1_gbm_t,AWT_1_svr_t,AWT_1_rf_t]
AWT_3_data_t = [AWT_3_lr_t,AWT_3_mlp_t,AWT_3_gbm_t,AWT_3_svr_t,AWT_3_rf_t]
AWT_5_data_t = [AWT_5_lr_t,AWT_5_mlp_t,AWT_5_gbm_t,AWT_5_svr_t,AWT_5_rf_t]
AWT_7_data_t = [AWT_7_lr_t,AWT_7_mlp_t,AWT_7_gbm_t,AWT_7_svr_t,AWT_7_rf_t]
AWT_9_data_t = [AWT_9_lr_t,AWT_9_mlp_t,AWT_9_gbm_t,AWT_9_svr_t,AWT_9_rf_t]

In [47]:
data_t = [
    mse_data_t,
    r2_data_t,
    mae_data_t,
    AWT_1_data_t,
    AWT_3_data_t,
    AWT_5_data_t,
    AWT_7_data_t,
    AWT_9_data_t
]

table = pd.DataFrame(data_t, columns=['Lineer reg', 'MLP', 'LightGBM','SVR','RF'])

table.index = ['MSE', 'R^2', 'MAE','AWT 0.1','AWT 0.3','AWT 0.5','AWT 0.7','AWT 0.9']
table.head(10)

,Lineer reg,MLP,LightGBM,SVR,RF
MSE,0.380183,0.369819,0.212013,0.383918,0.146403
R^2,0.453945,0.468831,0.695487,0.448581,0.789722
MAE,0.477323,0.469541,0.355896,0.474587,0.290159
AWT 0.1,0.135071,0.139021,0.192338,0.137046,0.227883
AWT 0.3,0.396130,0.406398,0.513823,0.411137,0.615324
AWT 0.5,0.610585,0.614929,0.750395,0.620458,0.838863
AWT 0.7,0.773697,0.778831,0.884676,0.770142,0.933649
AWT 0.9,0.869273,0.877172,0.949842,0.869668,0.971169
